In [1]:
# Testing Cell
import aviary.api as av
from aviary.api import Settings
from aviary.utils.doctape import glue_variable, check_value

str_problem_type = Settings.PROBLEM_TYPE
str_sizing = av.ProblemType.SIZING.value
str_alternate = av.ProblemType.ALTERNATE.value
str_fallout = av.ProblemType.FALLOUT.value


str_alternate_snippet = f'```\n{str_problem_type}, {str_alternate}\n```'
glue_variable('alternate_snippet', str_alternate_snippet, md_code=False)

fallout_snippet = f'```\n{str_problem_type}, {str_fallout}\n```'
glue_variable('fallout_snippet', fallout_snippet, md_code=False)

check_value(av.EquationsOfMotion.HEIGHT_ENERGY.value, 'height_energy')
check_value(av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM.value, '2DOF')


HEIGHT_ENERGY = av.EquationsOfMotion.HEIGHT_ENERGY
glue_variable('height_energy', HEIGHT_ENERGY.name, md_code=True)
TWO_DEGREES_OF_FREEDOM = av.EquationsOfMotion.TWO_DEGREES_OF_FREEDOM
glue_variable('2DOF', TWO_DEGREES_OF_FREEDOM.name, md_code=True)

file_path = av.get_path('examples/run_off_design_example.py').relative_to(av.top_dir.parent)
glue_variable('run_off_design_example.py', file_path, md_code=True)

file_path2 = av.get_path('examples/run_level2_example.py').relative_to(av.top_dir.parent)
glue_variable('run_level2_example.py', file_path2, md_code=True)

glue_variable('num_first', av.Aircraft.CrewPayload.NUM_FIRST_CLASS, md_code=False)
glue_variable('num_business', av.Aircraft.CrewPayload.NUM_BUSINESS_CLASS, md_code=False)
glue_variable('num_tourist', av.Aircraft.CrewPayload.NUM_TOURIST_CLASS, md_code=False)
glue_variable('num_pax', av.Aircraft.CrewPayload.NUM_PASSENGERS, md_code=False)
glue_variable('wing_cargo', av.Aircraft.CrewPayload.WING_CARGO, md_code=False)
glue_variable('misc_cargo', av.Aircraft.CrewPayload.MISC_CARGO, md_code=False)
glue_variable('cargo_mass', av.Aircraft.CrewPayload.CARGO_MASS, md_code=False)

glue_variable('operating_mass', av.Aircraft.Design.OPERATING_MASS, md_code=True)
glue_variable('empty_mass', av.Aircraft.Design.EMPTY_MASS, md_code=True)
glue_variable('cargo_container_mass', av.Aircraft.CrewPayload.CARGO_CONTAINER_MASS, md_code=True)
glue_variable('design_gross_mass', av.Mission.Design.GROSS_MASS, md_code=True)

file_path3 = av.get_path('models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv')

glue_variable('objectives_fuel', av.Mission.Objectives.FUEL, md_code=True)
glue_variable('objectives_range', av.Mission.Objectives.RANGE, md_code=True)
glue_variable('summary_gross_mass', av.Mission.Summary.GROSS_MASS, md_code=True)

```
settings:problem_type, alternate
```

```
settings:problem_type, fallout
```

`HEIGHT_ENERGY`

`TWO_DEGREES_OF_FREEDOM`

`aviary\examples\run_off_design_example.py`

...

`aircraft:crew_and_payload:cargo_container_mass`

`mission:design:gross_mass`

`mission:objectives:fuel`

`mission:objectives:range`

`mission:summary:gross_mass`

# Off-Design Missions

## Overview

Off-design missions are missions that take an already designed and sized aircraft and attempt to run different mission trajectories, different payload quantities, or both.

Off-design missions are enabled for the following mission types:

* {glue:md}`height_energy`
* {glue:md}`2DOF`

There are currently two types of off-design missions supported in Aviary: alternate and fallout.
For an alternate mission, the mission's target range and aircraft payload mass are inputs and the fuel mass required is solved for. 
Fallout missions, meanwhile, take the payload and fuel masses as inputs and determine the range of the aircraft. 

There are currently 3 different methods for running an off-design mission within Aviary. 

The first method is to take the input deck of an already sized aircraft and change its problem type to either `fallout` or `alternate`.    

```{note}
The user may need to revise some of the values in their input deck for the off-design mission.
Since the aircraft is not re-designed, it is assumed that provided inputs constitute a valid aircraft.
```
The second method is to run off-design missions in the same script used to solve the design mission.
An example of this is shown in {glue:md}`run_off_design_example.py`.

The third method is to run a sizing mission to design the aircraft, save the sizing information to a JSON file, then load the off-design mission in another script.  

```{note}
If the sizing mission did not converge to a valid aircraft design, any off-design analysis will be invalid even if the off-design missions themselves converged.
Therefore the validity of off-design analysis depends entirely on the validity of its source sizing mission.
```

## Off-Design from an already sized aircraft. 

The first method of running an off-design mission requires that the user has an input deck of a fully sized and valid aircraft.
ex: `aircraft_for_bench_FwFm.csv`.
The different problem types are expanded on in [Level 2](onboarding_level2), but a quick overview:
The problem types determine what the optimizer can control to find a valid solution. 
Sizing Missions allow the optimizer to control both the {glue:md}`summary_gross_mass` and {glue:md}`design_gross_mass` for the given mission and objective. 
Alternate Missions allow the optimizer to only control the {glue:md}`summary_gross_mass` for the mission.
Fallout Missions don't allow the optimizer to control either {glue:md}`summary_gross_mass` or {glue:md}`design_gross_mass` but allows the optimizer to extend the range until the summary matches the design.

This is done by adding one of the following lines to its csv file:

{glue:md}`fallout_snippet`

or

{glue:md}`alternate_snippet`


Once the problem type is specified, run Aviary as you would any other mission. 

## Sizing & off-design in the same script.

Firstly, we highly recommend users first read through and understand the [Level 2](onboarding_level2) onboarding guide before attempting an off-design mission. 

This is the Aviary Team's preferred method for running off-design missions as it allows the user to interrogate both the sizing and off-design missions together in the same script.
Off-design functionality within the script involves the transfer of sizing parameters using the `json_filename` into a new `AviaryProblem()` object.
The new `AviaryProblem()` then inputs the users new payload information, `phase_info` and `mission_mass` or `mission_range` depending on if the user is doing a fallout or alternate mission. 

Running an off-design mission first requires a sizing mission setup similar to {glue:md}`run_level2_example.py` then the appendment of `save_sizing_to_json()` to the `AviaryProblem`.
- `prob.save_sizing_to_json(json_filename = 'sizing_problem.json')` 

The `save_sizing_to_json()` argument saves the sizing mission's parameters to a JSON filename of the user's choosing where `sizing_problem.json` is the default filename.

- `prob.fallout_mission(json_filename = 'sizing_problem.json')` 

- `prob.alternate_mission(json_filename = 'sizing_problem.json')`

If an argument is left empty Aviary will assume it is the same as that of the sizing mission. 
Both off-design mission types take payload as an input parameter. 
The mission's payload is split up into the 3 passenger classes and the 2 cargo loadings. 
The respective aviary variables are then updated with the user's off-design passenger and cargo quantities.

```{note} 
As of v0.9.10 Aviary's off-design capabilities do not allow missions to be run with 0 passengers, so a minimum of 1 passenger must be specified. 
This is a known bug and is currently being investigated. 
```

| `Payload input`   | `Aviary Variable Name`    | `Description`
| ----------------- | ------------------------- | ----------------------------------------- |
| `num_first`       | {glue:md}`num_first`      | The number of first class passengers      |
| `num_business`    | {glue:md}`num_business`   | The number of business class passengers   |
| `num_tourist`     | {glue:md}`num_tourist`    | The number of tourist class passengers    |
| `num_pax`         | {glue:md}`num_pax`        | Total number of passengers                |
| `wing_cargo`      | {glue:md}`wing_cargo`     | Cargo carried in wing                     |
| `misc_cargo`      | {glue:md}`misc_cargo`     | Additional cargo carried in fuselage      |
| `cargo_mass`      | {glue:md}`cargo_mass`     | Total mass of as-flown cargo              |

```{note} 
Off-design missions **cannot** be ran with more passengers than the original sizing mission. 
For example, if {glue:md}`num_first` within the aviary inputs csv was set to 3, an off-design mission cannot be run where `num_first = 8` as the cabin for first class was sized for precisely 3 passengers.
```

The `phase_info` allows the user to specify a different phase information for an off-design mission trajectory. 
If a `phase_info` is not specified, the optimizer will assume it is the same as that of the sizing mission. 


### Alternate missions within the same script

Alternate missions take the `mission_range` argument and attempt to fly the sized aircraft with the user input payload and a given `phase_info` to that distance. 
The `mission_range` argument overwrites the `target_range` for {glue:md}`height_energy` and `mission_distance` for {glue:md}`2DOF` mission types.



Off-design missions are run with the assumption that the sizing mission's {glue:md}`design_gross_mass` is the maximum structural mass the aircraft can support. 
However, Aviary is capable of running a converging off-design missions with {glue:md}`design_gross_mass` that exceed that structural constraint. 
Therefore the user should check the {glue:md}`design_gross_mass` of the off-design to ensure the optimization is valid. 

### Fallout missions within the same script.

Fallout missions take the `mission_mass` argument that replaces {glue:md}`design_gross_mass` and attempts to fly the sized aircraft with the user input payload and a given `phase_info`  to as far as that aircraft is capable of going. 

```{note}
In cases where passengers or cargo are changed and the user has not specified a {glue:md}`cargo_container_mass` in the aviary input deck, off-design missions recalculates it in the off-design mission.
It is the only parameter within the operating mass that gets recalculated between off-design and sizing.
```

An example of a off-design mission being run within the same script:

In [2]:
%%capture

"""
This is an example of a sizing mission and alternate mission being ran in the same script. 
This is done with 
"""
import aviary.api as av

# inputs that run_aviary() requires
aircraft_data = 'models/aircraft/test_aircraft/aircraft_for_bench_FwFm.csv'
optimizer = 'SLSQP'
phase_info = av.default_height_energy_phase_info
max_iter = 15

# Set up and run avairy problem.
prob = av.AviaryProblem(verbosity=0)

# This is the default FwFm (Flops weights Flops mission) aircraft.
prob.load_inputs(aircraft_data, phase_info)

prob.check_and_preprocess_inputs()

prob.add_pre_mission_systems()

prob.add_phases()

prob.add_post_mission_systems()

prob.link_phases()

prob.add_driver(optimizer, max_iter=max_iter)

prob.add_design_variables()

prob.add_objective()

prob.setup()

prob.set_initial_guesses()

prob.run_aviary_problem()

# save the sizing mission to a JSON file
prob.save_sizing_to_json(json_filename='off_design_documentation_example.json')

# initialize alternate mission flying a much shorter distance.
# if payload values are unspecified, off-design will uses the values from the sizing mission.
# the mission_range argument is much shorter than the sizing mission's 1906 NM range.
prob.alternate_mission(
    json_filename='off_design_documentation_example.json',
    num_first=0,
    num_business=0,
    num_tourist=50,
    num_pax=50,
    wing_cargo=0,
    misc_cargo=0,
    cargo_mass=0,
    mission_range=1000,
    phase_info=phase_info,
    verbosity=0,
)

## Off-design missions separately

The third method in running an off-design mission requires saving the sizing information as a JSON file using `save_sizing_to_json()` then calls the JSON file into an off-design mission in another script.
Once saved, the `_load_off_design()` function can be used to create a new Aviary problem which will take the `json_filename` input along with 3 additional problem definition parameters `problem_type`, `equations_of_motion`, and `mass_method`.

- `problem_type`: Determines which type of off-design analysis the user would like to run. (alternate or fallout)
- `equations_of_motion`: Determines the type of trajectory calculations the user is running. ({glue:md}`height_energy` or {glue:md}`2DOF`)
- `mass_method`: Specifies how mass calculations are handled. 

Once defined, the `_load_off_design()` function will run an off-design mission of the specified `problem_type` and user input `phase_info`.
The user then can adjust payload quantities through the passenger class counts and cargo mass. 

- Alternate Missions: the user must specify a `mission_range`. 
- Fallout Missions: the user must specify a `mission_gross_mass`.


In [3]:
%%capture

"""
This is an example of running off-design mission directly form a saved sized aircraft JSON file
Note: If there were external subsystems in the original sized aircraft model 
these will NOT be transferred (since these are not saved in the JSON file).
"""

from aviary.interface.methods_for_level2 import _load_off_design
from aviary.variable_info.enums import LegacyCode
import aviary.api as av

optimizer = 'SLSQP'
phase_info = av.default_height_energy_phase_info
max_iter = 15
# Load aircraft and options data from provided sources

# To run an alternate mission, we need the sized aircraft JSON file from the sizing mission.
# Set ProblemType to ALTERNATE, and specify the mission range and payload mass.
# mission_gross_mass does nothing).

# Use specific _load_off_design function to define the off-design mission.
prob_alternate = _load_off_design(
    json_filename='off_design_documentation_example.json',
    problem_type=av.ProblemType.ALTERNATE,
    equations_of_motion=av.EquationsOfMotion.HEIGHT_ENERGY,
    mass_method=LegacyCode.FLOPS,
    phase_info=phase_info,
    num_first=0,
    num_business=0,
    num_tourist=50,
    num_pax=50,
    wing_cargo=0,
    misc_cargo=0,
    cargo_mass=0,
    mission_range=1000,
    verbosity=0,
)

# Run off-design mission the same way as any Level 2 aviary problem.
prob_alternate.check_and_preprocess_inputs()

prob_alternate.add_pre_mission_systems()

prob_alternate.add_phases()

prob_alternate.add_post_mission_systems()

prob_alternate.link_phases()

prob_alternate.add_driver(optimizer=optimizer, max_iter=max_iter)

prob_alternate.add_design_variables()

prob_alternate.add_objective()

prob_alternate.setup()

prob_alternate.set_initial_guesses()

prob_alternate.run_aviary_problem()

##

In [4]:
# Testing Cell
import os
import aviary.api as av
from aviary.utils.doctape import check_contains

# make sure off_design_example.py exists in aviary/examples folder
off_design_examples = av.get_path(os.path.join('examples'))
check_contains(
    ('run_off_design_example.py'),
    os.listdir(off_design_examples),
    error_string='{var} not in ' + str(off_design_examples),
    error_type=FileNotFoundError,
)

# make sure run_level2_example.py exists in aviary/examples folder
run_level2_examples = av.get_path(os.path.join('examples'))
check_contains(
    ('run_level2_example.py'),
    os.listdir(run_level2_examples),
    error_string='{var} not in ' + str(run_level2_examples),
    error_type=FileNotFoundError,
)